This file will ingest and clean the NYC 311 Data and convert it to a .csv file which will later be read into the '311_data_explore.ipynb' file

In [5]:
import requests
import pandas as pd
import numpy as np

Pull 311 Service Request Data for the year 2017 from the Socrata API 

In [19]:
offset = 0
LIMIT = 1000000 #max rows which can be pulled from the api with a single call
arr_of_df = [] #array which will hold multiple batches of data
URL_CONSTANT = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$WHERE=created_date between '2017-01-01T00:00:00.000' and '2017-12-31T23:59:59.999' &$limit=" + str({}) + " &$offset=" + str({})

print("loading in data...")

#pull data from api in batches of 1000000 (limit) until theres no more rows to pull
while True:
    apiData = requests.get(URL_CONSTANT.format(LIMIT, offset)).json()
    dfTemp = pd.DataFrame(data=apiData) #convert each batch of data into a dataframe so we can easily use the pandas concat method later
    arr_of_df.append(dfTemp)
    offset += LIMIT
    if (len(apiData)-LIMIT) >= 0: #check if the max limit was pulled from the api
        continue
    else: #near end of dataset, no need to send another request
        break
        
#concatenate all of the batches of data into one DataFrame
df_cityData = pd.concat(arr_of_df)

print("Data Download Complete!")

loading in data...
Data Download Complete!


Clean up "Unspecified" borough's using the dataset below (contains zipcode:borough pair):
https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php

I copy and pasted this data into an excel spreadsheet and saved it as an csv file. There are other ways to grab the data although I found this way to be the easiest and quickest.

*file already in github repo

In [20]:
#filter the "Unspecified" boroughs and their coordinating incident_zip from 'df_cityData'
df_filteredCityData = df_cityData[['borough', 'incident_zip']].dropna()
df_filteredCityData = df_filteredCityData.loc[df_filteredCityData['borough']=='Unspecified']

df_nycZipBorough = pd.read_csv('NYC_Zips.csv') #read in the dataset specified in the markdown above

#clean up the new data and format it correctly
df_nycZipBorough = df_nycZipBorough.dropna()
df_nycZipBorough['zipcode'] = df_nycZipBorough['zipcode'].apply(int).apply(str)#remove the '.0' from zips(.replace method was acting weird)
df_nycZipBorough['borough'] = df_nycZipBorough['borough'].str.upper().replace('STATEN', 'STATEN ISLAND')
df_nycZipBorough.rename(columns={'zipcode': 'incident_zip'} ,inplace=True)

#merge the two dataframes (containing the unspecified boroughs and the specified ones on the 'incident_zip'),
#dropping the "Unspecified" borough row in the process
nowSpecifiedBoroughs = pd.merge(df_filteredCityData['incident_zip'].to_frame(), df_nycZipBorough, on='incident_zip').drop_duplicates()

#merge the original cityData with the data containing the specified boroughs
df_cleanedCityData = pd.merge(df_cityData, nowSpecifiedBoroughs, how='left', on='incident_zip')

#Swap the "Unspecified" boroughs out with the 
df_cleanedCityData['borough_x']=df_cleanedCityData['borough_x'].replace('Unspecified', np.NaN)
df_cleanedCityData.borough_x.fillna(df_cleanedCityData.borough_y, inplace=True) #fill the now 'NaN' field with the specified borough value (borough_y)
df_cleanedCityData = df_cleanedCityData.drop(columns=['borough_y']) #drop the no longer needed row
df_cleanedCityData.rename(columns={'borough_x': 'borough'} ,inplace=True)

print("Clean-up Complete!")

Clean-up Complete!


In [21]:
print("converting to csv file...")
#convert dataframe into a csv file and save it to the current directory (should be the same as your jupyter notebook)
df_cleanedCityData.to_csv('2017_NYC_Data.csv', index=False)
print("Conversion Complete!")
#note: there is an opportunity to paralize the fetching step using threading

converting to csv file...
Conversion Complete!
